# Sentiment Classifer
The goal is to make a prograom that can guess the sentiment of a given sentence bassed on others. My dataset is that of Reddit comments and I am using the number of upvotes as the mesurement of sentiment.

In [86]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential
from tensorflow.keras import layers

Import File

In [87]:
df = pd.read_csv('train-balanced-sarcasm.csv')
print('File Length: ', len(df))

File Length:  1010826


In [88]:
df.sample(5)

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
962320,0,Faster.,pash1k,CFB,3,3,0,2013-10,2013-10-04 05:23:17,"""Almost like a jet taking off... from an aircr..."
174624,0,"Correct, but if you want to crack a walnut a p...",thedugong,worldnews,1,-1,-1,2016-10,2016-10-11 23:19:18,So are nuclear ICBMs.
595587,1,It was the best name they had that started wit...,LUSTY_BALLSACK,pcmasterrace,1,1,0,2015-07,2015-07-07 05:47:27,why companys always throw away cool project na...
493873,1,You obviously didn't try hard enough.,Colonel_of_Wisdom,headphones,5,5,0,2016-01,2016-01-23 03:26:27,Sadly they don't fit onto the q701
270816,0,"Your stunning lack of details, insights and fa...",Funklestein,todayilearned,0,0,0,2016-07,2016-07-01 17:58:40,And now I know you don't know what you're talk...


In [89]:
df = pd.DataFrame(df.sample(35000))
df = df[['comment', 'score']]
df = df.dropna()

print(df.sample(10))
print('Length of the Data Field: ', len(df))

                                                  comment  score
666543     "What's wrong with complimenting her body!!??"    192
165755  No no obviously they should have suck it in an...      6
863251                                        marssantoso      1
446770  No it was sheer brilliance and great game read...      1
295425  It's ok, guys, Erdogan is just taking some pre...      2
934281         TL;DR kid gets what he wants for Christmas      1
31108                           I thought the same thing.      6
88900                                    Alllllmost there      1
919501                          Are you an authoritarian?      6
675009                         Chestbrah lookin' massive!      8
Length of the Data Field:  34998


This is my function I am using to decide sentiment. Current the bounds are at -2 and 10 for negitive and positive respectivly. Ther values have to be fairly low due to the fact that such a large portion of the dataset contains values around or at 0.

In [90]:
def sentiment(val):
    if val > 10:
        return 2
    if val < -2:
        return 0
    return 1

In [91]:
df['score'] = df['score'].apply(sentiment)
print(df.sample(10))

                                                  comment  score
510166               I was born 9 months after a Tuesday.      1
545416               Kira Yamato vs ~~Tusk~~ Kira Yamato.      1
702925  Yeah, who cares about the people that actually...      1
408173                    U mean every Sunday of my life?      1
367612  TRP claims to have predicted that, you hyperga...      1
678219  sigh... if only we had european style hate spe...      0
564128  You guys look like you blow a load onto each o...      1
81540   You're grossing me out with your justification...      1
246371                                    GGG Slave Owner      1
821384   Well, *that's* a helpful and insightful comment.      1


In [92]:
comments = df['comment'].iloc[:30000]
test_comments = df['comment'].iloc[30000:]
scores = df['score'].iloc[:30000]
test_scores = df['score'].iloc[30000:]

In [93]:
tokenizer = Tokenizer(num_words=8000)

In [94]:
tokenizer.fit_on_texts(comments)

In [95]:
comments = tokenizer.texts_to_sequences(comments)

print(type(comments[0][0]))

<class 'int'>


In [96]:
comments = tokenizer.sequences_to_matrix(comments)

In [97]:
print(type(comments))
print(comments.dtype)

<class 'numpy.ndarray'>
float64


In [98]:
test_comments = tokenizer.texts_to_sequences(test_comments)
test_comments = tokenizer.sequences_to_matrix(test_comments)

Creation of the learning model

In [99]:
model = Sequential()

In [100]:
model.add(layers.Dense(700, activation='relu'))
model.add(layers.Dropout(0.1))

In [101]:
model.add(layers.Dense(400, activation='relu'))
model.add(layers.Dropout(0.2))

In [102]:
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dropout(0.4))

In [103]:
model.add(layers.Dense(3, activation='sigmoid'))

In [104]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [105]:
print('Num of GPUS:', len(tf.config.experimental.list_physical_devices('GPU')))

Num of GPUS: 0


In [106]:
comment_y = keras.utils.to_categorical(scores,3)
comment_x = comments

In [107]:
model.fit(comment_x, comment_y,
         batch_size=35,
         epochs=5,
         validation_split=0.2,
         shuffle=True)

Train on 24000 samples, validate on 6000 samples
Epoch 1/5
24000/24000 [==============================] - 23s 944us/sample - loss: 0.5332 - accuracy: 0.8542 - val_loss: 0.4912 - val_accuracy: 0.8568
Epoch 2/5
24000/24000 [==============================] - 27s 1ms/sample - loss: 0.4534 - accuracy: 0.8543 - val_loss: 0.5213 - val_accuracy: 0.8565
Epoch 3/5
24000/24000 [==============================] - 23s 956us/sample - loss: 0.2276 - accuracy: 0.9204 - val_loss: 0.7098 - val_accuracy: 0.8162
Epoch 4/5
24000/24000 [==============================] - 37s 2ms/sample - loss: 0.0742 - accuracy: 0.9768 - val_loss: 1.1801 - val_accuracy: 0.8297
Epoch 5/5
24000/24000 [==============================] - 27s 1ms/sample - loss: 0.0415 - accuracy: 0.9881 - val_loss: 1.3305 - val_accuracy: 0.8232


I feel that there is deffiently something wrong with my model. The fact that it is currently scoring a 98% accuracy I find a little strange. The one reason I can see is that because of the way that I devide my data. The model could be looking for just a few keywords that it can elimenate. Either that or some user error on my behalf.

In [108]:
test_scores = keras.utils.to_categorical(test_scores,3)
test_loss, test_acc = model.evaluate(test_comments, test_scores, verbose=2)

print('\nTest accuracy:', test_acc)

4998/1 - 2s - loss: 0.6454 - accuracy: 0.8211

Test accuracy: 0.8211284


# Conclusions
After checking the accuarcey I am a little more confident in my model. I never really expected this to be a ground breaking data set. Any dataset that looks at comments espiceally those on Reddit and takes the Karma into account need to do some searous vetting before hand. Most of the time a comment will fail to gain any traction and on a site where everything is displayed based on popularity at a given time it is very easy for information to be lost. I've run the traing several times and have gotten simular results to where the training gets an accuracy score of around 98% and the actual test accuarcy is closer to 80%.

TLDR: Timing is more important to successful comments on Reddit then content.